In [1]:
import numpy as np
import string

In [2]:
class Node:
    def __init__(self, data, *args):
        self.data = data
        self.children = []
        self.name = ""
        self.classname = "Node"
        self.complete = True
        self.noperator = 0
        
        if len(args)>0:
            self.name=args[0]
    
    def __str__(self):
        s = self.classname[:1] + "("+str(self.data)
        if self.name!="":
            s += ", \"" + self.name +"\""
        return s + ")"
    def __repr__(self):
        s = self.classname + "("+str(self.data)
        if self.name!="":
            s += ", \"" + self.name +"\""
        return s + ")"
    
    def __eq__(self, other):
        return self.name == other.name
    
    def printTreeNodes(self):
        if self.noperator == 0: #variable
            print(self)
        elif self.noperator == 1: #Operador unario
            print(self)
            self.children[0].printTreeNodes()
        elif self.noperator == 2: #Operador binario
            self.children[0].printTreeNodes()
            print(self)
            self.children[1].printTreeNodes()
        else: #Paréntesis
            print(self)
            self.children[0].printTreeNodes()
            print(")")
            
    def __printExpressionRec__(self):
        if self.noperator == 0: #variable
            print(self.name,end="")
        elif self.noperator == 1: #Operador unario
            print(self.name,end="")
            self.children[0].__printExpressionRec__()
        elif self.noperator == 2: #Operador binario
            self.children[0].__printExpressionRec__()
            print(self.name,end="")
            self.children[1].__printExpressionRec__()
        else: #Paréntesis
            print("(", end="")
            self.children[0].__printExpressionRec__()
            print(")", end="")
    def printExpression(self):
        self.__printExpressionRec__()
        print()

    def isComplete(self):
        return self.complete
    
    def setComplete(self):
        self.complete = True
        
    def insert(self, N):
        self.children.append(N)
    
    def function(self):
        return self.data

In [3]:
class Parentesis(Node):
    def __init__(self, data, *args):
        super().__init__(data, *args)
        self.classname = "Parentesis"
        self.complete = False
        self.noperator = -1
    
    def function(self):
        if len(self.children)>=1:
            return self.children[0].function()
        else:
            return None
    
    def insert(self, N):
        if self.isComplete():
            print("Error, inserting on complete node 1.")
        super().insert(N)
        if len(self.children)>=1:
            self.setComplete()

In [4]:
class Conjuncion(Node):
    def __init__(self, data, *args):
        super().__init__(data, *args)
        self.classname = "Conjuncion"
        self.name = "^"
        self.complete = False
        self.noperator = 2
    
    def function(self):
        return children[0].function() and children[1].function()
    
    def insert(self, N):
        if self.isComplete():
            print("Error, inserting on complete node 2.")
        super().insert(N)
        if len(self.children)>=2:
            self.setComplete()

In [5]:
class Disyuncion(Node):
    def __init__(self, data, *args):
        super().__init__(data, *args)
        self.classname = "Disyuncion"
        self.name = "|"
        self.complete = False
        self.noperator = 2
        
    def function(self):
        return children[0].function() or children[1].function()
    
    def insert(self, N):
        if self.isComplete():
            print("Error, inserting on complete node 3.")
        super().insert(N)
        if len(self.children)>=2:
            self.setComplete()

In [6]:
class Negacion(Node):
    def __init__(self, data, *args):
        super().__init__(data, *args)
        self.classname = "Negacion"
        self.name = "¬"
        self.complete = False
        self.noperator = 1
    
    def function(self):
        return not children[0].function()
    
    def insert(self, N):
        if self.isComplete():
            print("Error, inserting on complete node 4.")
        super().insert(N)
        if len(self.children)>=1:
            self.setComplete()

In [7]:
class Implicacion(Node):
    def __init__(self, data, *args):
        super().__init__(data, *args)
        self.classname = "Implicacion"
        self.name = "->"
        self.complete = False
        self.noperator = 2
    
    def function(self):
        if self.children[0].function()==True and self.children[1].function()==False:
            return False
        else:
            return True

    def insert(self, N):
        if self.isComplete():
            print("Error, inserting on complete node 5.")
        super().insert(N)
        if len(self.children)>=2:
            self.setComplete()    

In [8]:
class DobleImplicacion(Node):
    def __init__(self, data, *args):
        super().__init__(data, *args)
        self.classname = "DobleImplicacion"
        self.name = "<->"
        self.complete = False
        self.noperator = 2
    
    def function(self):
        return self.children[0].function()==self.children[1].function()==False

    def insert(self, N):
        if self.isComplete():
            print("Error, inserting on complete node 6.")
        super().insert(N)
        if len(self.children)>=2:
            self.setComplete()    

In [13]:
def string2Proposition(str_prop):
    Letras = string.ascii_lowercase + string.ascii_uppercase
    valid_chars = "|^¬ <->()"+Letras
    Variables = []
    node_stack = []
    j = None
    i=0
    while i < len(str_prop):        
        if str_prop[i] == ' ':
            i+=1
            continue
        
        #Agregar el nodo al stack, sea cual sea
        if str_prop[i]=='(':
            node_stack.append(Parentesis(None))
            i+=1
            continue
        
        if str_prop[i] in Letras:
            if str_prop[i] not in map(lambda x: x.name, Variables):
                Variables.append(Node(None, str_prop[i]))
            j = Variables.index(Node(None, str_prop[i]))   
            node_stack.append(Variables[j])
        
        if str_prop[i] == '¬':
            node_stack.append(Negacion(None))
            
        if str_prop[i] == '^':
            if len(str_prop)==1 or not node_stack[-1].isComplete():
                print("String error 1")
                return None
            node_stack.append(Conjuncion(None))
    
        if str_prop[i] == '|':
            if len(str_prop)==1 or not node_stack[-1].isComplete():
                print("String error 2")
                return None
            node_stack.append(Disyuncion(None))
        
        if str_prop[i] == '-':
            if (i >= len(str_prop)-1) or str_prop[i+1]!='>':
                print("String error 3")
                return None
            node_stack.append(Implicacion(None))
            i+=1 #Porque este operador es de dos caracteres
        
        if str_prop[i] == '<':
            if (i >= len(str_prop)-2) or str_prop[(i+1):(i+1+2)]!="->":
                print("String error 4")
                return None
            node_stack.append(DobleImplicacion(None))
            i+=2 #Porque este operador es de tres caracteres
        
        
        #Caso especial: Cierre de paréntesis
        if str_prop[i] == ')':
            #Asegurar que no haya habido algún error con la cadena
            if type(node_stack[-2])!= Parentesis:
                print("String error 5")
                return None
            #Agregar el contenido del paréntesis dentro del nodo paréntesis
            node_stack[-2].insert(node_stack[-1])
            node_stack.remove(node_stack[-1])
            
            #Si es que hay un operador antes del paréntesis, agregar el contenido del paréntesis al operador.
            if (len(node_stack)>1) and (node_stack[-2].noperator >0):
                node_stack[-2].insert(node_stack[-1])
                node_stack.pop()
            i+=1
            continue
        
        #Revisar qué hacer con el elemento que se acaba de añadir al stack
        #Variables:
        if node_stack[-1].noperator == 0:
            #Casos en los que no hay que meter al nodo dentro de otro todavía
            if (len(node_stack)==1) or (type(node_stack[-2])==Parentesis):
                i+=1
                continue
            #Casos en los que hay un operador antes de la variable
            node_stack[-2].insert(node_stack.pop())
            i+=1
            continue
            
        #Operadores unarios:
        if (node_stack[-1].noperator == 1) and (type(node_stack[-1]!=Parentesis)):
            i+=1
            continue #No hay que hacer nada ya que no se vincula con lo que hay antes de él
            
        #Operadores binarios:
        if node_stack[-1].noperator == 2:
            node_stack[-1].insert(node_stack[-2])
            del node_stack[-2]
        
        i+=1
        
    print("    ", len(node_stack), node_stack)
    return Variables, node_stack[0]

In [15]:
(v,t) = string2Proposition("(a^b)->c")
t.printExpression()
print(v)

     1 [Implicacion(None, "->")]
(a^b)->c
[Node(None, "a"), Node(None, "b"), Node(None, "c")]
